# **使用雲端資料**

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)
!mkdir -p drive
!google-drive-ocamlfuse drive

Mounted at /content/drive
/bin/bash: google-drive-ocamlfuse: command not found


# **Install**

In [ ]:
!pip install gensim
!pip install numpy 
!pip install sklearn

# **Import&path**

In [ ]:
import numpy as np
import pandas as pd
import copy
from gensim.models import Word2Vec
import keras
from numpy import dot
import math
from sklearn.preprocessing import normalize

file_path = './drive/My Drive/IR/HW1/'

# **Read file**

In [ ]:
## list
query_list = []
doc_list = []
all_query = []
all_doc = []
##query list
with open(file_path+'query_list.txt','r') as f:
  query = f.readlines()
  for i in range(len(query)):
    query_list.append(query[i].strip('\n'))

## doc list
with open(file_path+'doc_list.txt','r') as f:
  doc = f.readlines()
  for i in range(len(doc)):
    doc_list.append(doc[i].strip('\n'))
##all query
for i in range(len(query_list)):
  with open(file_path+'./queries/'+str(query_list[i])+'.txt','r') as f:
    q_content = f.readlines()
    all_query.append(q_content)
## all doc
for i in range(len(doc_list)):
  with open(file_path+'./docs/'+str(doc_list[i])+'.txt','r') as f:
    d_content = f.readlines()
    all_doc.append(d_content)
print(query_list)
print(doc_list)
print(all_query[0])
print(all_doc[0])

['301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350']
['FBIS3-10082', 'FBIS3-10231', 'FBIS3-10243', 'FBIS3-10285', 'FBIS3-10291', 'FBIS3-10302', 'FBIS3-10397', 'FBIS3-10433', 'FBIS3-10451', 'FBIS3-10464', 'FBIS3-10491', 'FBIS3-10501', 'FBIS3-10506', 'FBIS3-10535', 'FBIS3-10551', 'FBIS3-10552', 'FBIS3-10622', 'FBIS3-10632', 'FBIS3-10633', 'FBIS3-10697', 'FBIS3-10698', 'FBIS3-10721', 'FBIS3-10753', 'FBIS3-10805', 'FBIS3-10910', 'FBIS3-10937', 'FBIS3-11003', 'FBIS3-11058', 'FBIS3-11069', 'FBIS3-11095', 'FBIS3-11099', 'FBIS3-11106', 'FBIS3-11107', 'FBIS3-11210', 'FBIS3-11405', 'FBIS3-11465', 'FBIS3-11502', 'FBIS3-11525', 'FBIS3-11553', 'FBIS3-11628', 'FBIS3-11634', 'FBIS3-11684', 'FBIS3-11961', 'FB

In [ ]:
doc_list.remove('LA072189-0048')
for i in range(len(doc_list)):
  if (doc_list[i]=='LA072189-0048'):
    print(i)
print(all_doc[3643])
del all_doc[3643:3644]
print(len(all_doc))

[]
4190


# **TF-IDF**

In [ ]:
##會return 所有得分跟所有欄位
def my_tfidf(all_input):
  column_name = []
  all_data_split = [] ##分割字串
  for i in range(len(all_input)):
    all_data_split.append(all_input[i][0].split())
  ## 讀入並掃次數
  voc_dic = {}
  score_dict_list = [] ##拿來存tf的所有權重

  example_dic = {} ##拿來乘的空dict
  for i in range(len(all_data_split)): ##幾篇文章 
    score_dic = {} ##算tf用的
    updated = {} ##已經算過的字
    for j in range(len(all_data_split[i])): ##所有的字
      if (voc_dic.get(all_data_split[i][j]) != None): ##已經存在
        if (all_data_split[i][j] not in updated):   ##這篇還沒看到
          voc_dic[all_data_split[i][j]] += 1.0
          updated[all_data_split[i][j]] = 1.0
      else:        ##沒有發現過這個字
        voc_dic[all_data_split[i][j]] = 1.0   #初始化
        updated[all_data_split[i][j]] = 1.0   #加入更新過的陣列
      if score_dic.get(all_data_split[i][j]) == None:
        score_dic[all_data_split[i][j]] = 1.0 ##建一個
      else:
        score_dic[all_data_split[i][j]] += 1.0 ##計數加一
      example_dic[all_data_split[i][j]] = 0
    score_dict_list.append(score_dic) ##存了所有tf有值的分數
  column_name = list(voc_dic.keys())
  ## idf
  idf_dic = {} ##idf權重
  N = len(all_input) ##總共幾篇
  for i in voc_dic:
    idf_dic[i] = math.log(1+((N+1)/(voc_dic[i]+1))) ##sklearn文檔中的smooth_idf
  result = []
  for i in range(N):
    for value in score_dict_list[i]:
      example_dic[value] =idf_dic[value]*(1+math.log(score_dict_list[i][value]))
    score = list(example_dic.values())
    result.append(score)
    for value in score_dict_list[i]:
      example_dic[value] = 0.0
  print('before norm')
  result_score = normalize(result,norm ='l2')
  return result_score,column_name

In [ ]:
## 所有文檔 包括query跟doc
all_input = all_query + all_doc
result_score,column_name = my_tfidf(all_input)
df_tfidf = pd.DataFrame(result_score,columns=column_name, index=(query_list+doc_list)) ##弄成dataframe
df_tfidf

intern     organ     crime  ...    reigel     verna  caraccilo
301            0.389644  0.544460  0.742792  ...  0.000000  0.000000   0.000000
302            0.164970  0.230517  0.000000  ...  0.000000  0.000000   0.000000
303            0.115167  0.160926  0.000000  ...  0.000000  0.000000   0.000000
304            0.103701  0.144904  0.000000  ...  0.000000  0.000000   0.000000
305            0.099858  0.139534  0.000000  ...  0.000000  0.000000   0.000000
...                 ...       ...       ...  ...       ...       ...        ...
LA123090-0026  0.000920  0.000759  0.000000  ...  0.000000  0.000000   0.000000
LA123189-0136  0.000920  0.000759  0.000000  ...  0.000000  0.000000   0.000000
LA123190-0040  0.000920  0.000759  0.001035  ...  0.000000  0.000000   0.000000
LA123190-0062  0.000920  0.000759  0.000000  ...  0.000000  0.000000   0.000000
LA123190-0065  0.000919  0.000759  0.001035  ...  0.003186  0.003186   0.003186

[4240 rows x 59680 columns]

# **處理成斷詞的格式**

In [ ]:
query_data = []
doc_data = []
for i in range(len(all_query)):
  query_data.append(all_query[i][0].split())

for i in range(len(all_doc)):
  if len(all_doc[i]) != 0:
    doc_data.append(all_doc[i][0].split())
print(query_data[0])
print(doc_data[0])

['intern', 'organ', 'crime']
['languag', 'f', 'p', '105', 'spanish', 'f', 'articl', 'type', 'bfn', 'text', 'santa', 'fe', 'de', 'bogota', '28', 'feb', 'dpa', 'today', 'colombian', 'prosecutor', 'gener', 'gustavo', 'de', 'greiff', 'said', 'u', 'govern', 'interest', 'suppli', 'evid', 'condemn', 'chief', 'drug', 'traffick', 'mafia', 'trust', 'colombian', 'justic', 'manner', 'de', 'greiff', 'reaffirm', 'difficulti', 'offic', 'encount', 'attempt', 'collect', 'evid', 'unit', 'state', 'main', 'capo', 'cali', 'cartel', 'sever', 'express', 'desir', 'surrend', 'collabor', 'author', 'colombia', 'desert', 'unit', 'state', 'interest', 'collabor', 'colombian', 'justic', 'although', 'judici', 'exchang', 'agreement', 'de', 'greiff', 'point', 'prosecutor', 'gener', 'ad', 'u', 'author', 'withhold', 'evid', 'reveal', 'later', 'mafioso', 'captur', 'countri', 'without', 'take', 'account', 'sever', 'think', 'surrend', 'colombia', 'opinion', 'director', 'colombian', 'justic', 'system', 'govern', 'washington'

In [ ]:
##找出Index
def find_index(input):
  max = -1.0
  id = 0
  for i in range(len(input)):
    if (input[i] > max):
      max = input[i]
      id = i
  return id

In [ ]:
ans_list = []
for i in range(len(query_data)): ##總共幾個query
  score_list = []       ##每個cos_sim的得分
  query_ans_list = []   ##排序輸出結果
  query_vec = np.zeros((len(query_data[i]))) ##每個query都有N個字當作n維向量
  doc_vec = np.zeros((len(doc_data),len(query_data[i]))) ##然後把doc的每個維度讀出來
  for j in range(len(query_data[i])): ## query有幾個詞
    query_vec[j] = df_tfidf.iloc[i][query_data[i][j]] ##query中每個字的分數存成vec
    for k in range(len(doc_data)):    ##總共幾篇doc
      doc_vec[k][j] = df_tfidf.iloc[k+50][query_data[i][j]] ##拿query中的字去看每篇doc的得分 弄成vec
  ##都存好開始算每個分數
  for j in range(len(doc_data)):
    cos_sim = 0
    for k in range(len(query_data[i])):
      cos_sim += dot(dot(query_vec[k],doc_vec[j][k]),query_vec[k]) ## query_vec的權重平方
    cos_sim /= len(query_data[i])
    score_list.append(cos_sim)
  for k in range(len(doc_data)):##用function找出從高到低的分數並記錄
    pos = find_index(score_list)
    query_ans_list.append(doc_list[pos])
    score_list[pos] = -2
  ans_list.append(query_ans_list)

# **弄出結果**

In [ ]:
error_name = 'LA072189-0048' ##錯誤的檔案 把它塞在最低分
with open(file_path+'hw1_result_mytfidf_new.txt','w') as f:
  f.write('Query,RetrievedDocuments\n')
  for i in range(50): 
    f.write(query_list[i])
    f.write(',')
    for j in range(len(ans_list[0])):
      f.write(ans_list[i][j])
      f.write(' ')
    f.write(error_name)
    f.write('\n')
  f.close()
